### 简明的概述

1. 读取train.txt 中存在的边，加入到NETWOKX 构建的graph（无向图）中。无向图中所有的边都视为真数据（positive data）
2. 根据无向图中的点，随机抓取两个点构成一条边，如果该边在图中不存在，则视为伪数据（negative data）,抓取出和真数据数量一致的伪数据
3. 从图中提取真伪数据的特征（各种图特征），组成训练数据，然后训练一个3层的Neural Networks。
4. 把test中的点从图中提取特征，用Neural Network 预测。

Note：这里有一个trick，就是为了训练的点不是乱选的，所有参与训练的边，至少有一个点是从test_data中出现的。这是一个出于直觉的设计，并没有验证时一定有用的。


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import math
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import math
import pandas as pd
import pickle
import math
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# 读写操作
dirname = 'data/'
def save_obj(obj, name ):
    with open(dirname+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( dirname + name + '.pkl', 'rb') as f:
        return pickle.load(f)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# train.TXT中有9130行数据，但是存在很多重复的点，去除重复后只有3767个点和32068条边

In [ ]:
from tqdm import tqdm
import random

# 读取train.txt file
with open("data/train.txt", "r") as f:
     train_data = f.readlines()
# SourceNodes 是全部的起点，Nodes和Edges是全部的点和边
SourceNodes = []
Nodes = []
Edges = []
for i in tqdm(range(len(train_data))):
    nodes_list = [int(n) for n in train_data[i].split()]
    SourceNodes.append(nodes_list[0])
    for node in nodes_list:
        Nodes.append(node)
    for source in nodes_list:
        for sink in nodes_list:
            if source!=sink:
                Edges.append((source, sink))
# 生成Nodes和Edges，直接从train.txt 读取就可以

100%|██████████| 9310/9310 [00:00<00:00, 230217.14it/s]


In [ ]:
print("Before removing duplicates, we have: ")
print('Nodes: ',len(Nodes),'\tEdges:', len(Edges))

Before removing duplicates, we have: 
Nodes:  25037 	Edges: 59236


In [ ]:
setofEdges = set(Edges)
setofNodes = set(Nodes)
print("After removing duplicates, we have: ")
print('Nodes: ',len(setofNodes),'\tEdges:', len(setofEdges))

After removing duplicates, we have: 
Nodes:  3767 	Edges: 32068


In [ ]:
# 生成正面数据

'''
Edges就32068条，全部作为正面数据好了

'''
pos_data = []
for elem in tqdm(setofEdges):
    pos_data.append([(elem[0], elem[1]), 1])

print(len(pos_data))

100%|██████████| 32068/32068 [00:00<00:00, 379199.84it/s]

32068


In [ ]:
# 生成负面数据

neg_data = []
for source in tqdm(setofNodes):
    for sink in random.sample(setofNodes, 3767):
        if source!=sink and not (source,sink) in setofEdges:
            neg_data.append([(source, sink), 0])


print('generated ',len(neg_data),' fake edges.')
neg_data = random.sample(neg_data, 32068)
print('To get the same size sample of true edges, we random pick ',len(neg_data))

100%|██████████| 3767/3767 [00:29<00:00, 126.29it/s]


generated  14154454  fake edges.
To get the same size sample of true edges, we random pick  32068


In [ ]:
save_obj(pos_data, 'pos_data')
save_obj(neg_data, 'neg_data')
print(len(pos_data), len(neg_data))
print(len(pos_data[1]), len(neg_data[1]))
print(pos_data[1], neg_data[1])
# We set training data as the concatenation of pos_data and neg_data
training_edges = []
for i in range(32068):
    training_edges.append(pos_data[i])
    training_edges.append(neg_data[i])

32068 32068
2 2
[(19, 1917), 1] [(3947, 1600), 0]


In [ ]:
# train.txt 文件已经读取， pos_data 和 neg_data 也已经读取
print('Nodes number:\t','Edges number:')
print(len(setofNodes), '\t',len(setofEdges), '\n')
print('posdata_length: ', 'negdata_length:')
print(len(pos_data),'\t', len(neg_data), '\n')

print('first 10 pieces of pos_data and neg_data:')
print(pos_data[:10],'\n')
print(neg_data[:10])



Nodes number:	 Edges number:
3767 	 32068 

posdata_length:  negdata_length:
32068 	 32068 

first 10 pieces of pos_data and neg_data:
[[(1410, 1872), 1], [(19, 1917), 1], [(1073, 3626), 1], [(151, 3004), 1], [(148, 3935), 1], [(107, 3616), 1], [(3677, 2457), 1], [(1527, 3974), 1], [(1162, 92), 1], [(602, 2421), 1]] 

[[(3490, 3521), 0], [(3947, 1600), 0], [(2892, 175), 0], [(3301, 232), 0], [(2165, 3948), 0], [(2369, 881), 0], [(2289, 568), 0], [(4064, 2055), 0], [(1836, 861), 0], [(786, 2436), 0]]


In [ ]:
G = nx.Graph()
G.add_nodes_from(Nodes)
G.add_edges_from(setofEdges)
# Edges and Nodes are all from train.txt (pos_data)
print('Graph has been built!')

Graph has been built!


In [ ]:
# 保存G对象到本地
save_obj(G, 'graph')
print('graph has been saved')

graph has been saved


In [ ]:
'''
接下来生成edges_of_all_test_data_related

首先需要提取test.public 中所有的点，存储为 nodes_in_test_data

然后根据nodes_in_test_data 生成 edges_of_all_test_nodes_related
'''

with open("data/test-public.csv", "r") as f:
     test_data = f.readlines()

# 提取test_data 后删除第一行的ID
test_data = test_data[1:]
# 测试
for i in range(len(test_data)):
    test_data[i] = [int(elem) for elem in test_data[i].split(',')]
print(test_data[1])

[2, 0, 2956]


In [ ]:
# 提取test_data 中所有点
nodes_in_test_data = []
for line in test_data:
    source = line[1]
    sink = line[2]
    nodes_in_test_data.append(int(source))
    nodes_in_test_data.append(int(sink))


temp = nodes_in_test_data
temp_set = set(temp)
temp = list(temp_set)
nodes_in_test_data = temp
# 生成出nodes_in_test_data 后用hash set去重
# 可以发现大量点是重复的
print('Test data has ', len(nodes_in_test_data), 'nodes without duplicates')

Test data has  2314 nodes without duplicates


In [ ]:
# 要把test中不存在于training data的点也加入到Graph中，这样才能生成完整的pre-features
G.add_nodes_from(nodes_in_test_data)

In [ ]:
'''
debug: 为什么所有的边都被保存为了inbound？

solution:  代码从trian.txt中读取了Edges，错在“直接把Edges转化为hash set 然后判断(e, node)是否存在”

'''

'''
生成pre_features: pre_features 需要包含哪些特征？

1. neighbors number 
2. log(nei_number)
3. all_neighbors
4. inbound
5. inbound_num
6. outbound
7. outboud_num
    
'''
pre_features = {}
count = 0
for node in tqdm(list(G.nodes)):
    neig = sorted(nx.all_neighbors(G, node))                          # 3
    num_neig = len(neig)                 # 1
    log_neig = (1. / math.log(num_neig+1)) if num_neig != 0 else 0    # 2

    
    inbound = []
    outbound = []
    for e in list(neig):
        if (e,node) in setofEdges:
            inbound.append(e)
        if (node,e) in setofEdges:
            outbound.append(e)
    if len(inbound) == len(outbound):
        count += 1
    pre_features[node] = [num_neig, log_neig, neig, inbound, len(inbound), outbound, len(outbound)]
print(count)
# It's a undirectional Graph, what's the mean to differ inbound and outbound of nodes.

100%|██████████| 3816/3816 [00:00<00:00, 94762.96it/s]

3816


In [ ]:
print(pre_features[1864])
save_obj(pre_features, 'pre_features')

[0, 0, [], [], 0, [], 0]


In [ ]:
import pandas as pd
BasicFeatures = load_obj('pre_features')
# in-degree statistics
df = pd.DataFrame(BasicFeatures)
df =  df.T
df.rename(columns={0: 'num_of_neighbours',
                   1: 'log_num_of_neighbours',
                  2: 'list_of_neighbours',
                  3: 'list_of_in_neighbours',
                  4: 'num_of_in_neighbours',
                  5: 'list_of_out_neighbours',
                  6: 'num_of_out_neighbours'}, inplace=True)
df.loc[[2]]

,num_of_neighbours,log_num_of_neighbours,list_of_neighbours,list_of_in_neighbours,num_of_in_neighbours,list_of_out_neighbours,num_of_out_neighbours
2,24,0.310667,"[51, 64, 585, 605, 932, 1146, 1230, 1348, 1389...","[51, 64, 585, 605, 932, 1146, 1230, 1348, 1389...",24,"[51, 64, 585, 605, 932, 1146, 1230, 1348, 1389...",24


In [ ]:
"""
生成edges_of_all_test_nodes_related:
    思路是提取所有的训练数据的edge都是和test_data中的点相关的
"""
from tqdm import tqdm 
edges_of_all_test_nodes_related = []
nodes_related = []
for line in tqdm(setofEdges):
    (node1, node2) = line
    if node1 in nodes_in_test_data:
        nodes_related.append(node1)
    if node2 in nodes_in_test_data:
        nodes_related.append(node2)
    if (node1 in nodes_in_test_data) or (node2 in nodes_in_test_data):
        edges_of_all_test_nodes_related.append((node1, node2))

print('edges_related: ',len(edges_of_all_test_nodes_related))
print(edges_of_all_test_nodes_related[:10])
nodes_related = set(nodes_related)
print('\nnodes related: ', len(nodes_related))

save_obj(edges_of_all_test_nodes_related, 'edges_of_all_test_nodes_related')

100%|██████████| 32068/32068 [00:02<00:00, 15433.33it/s]

edges_related:  29868
[(1410, 1872), (19, 1917), (1073, 3626), (151, 3004), (148, 3935), (107, 3616), (3677, 2457), (1527, 3974), (1162, 92), (602, 2421)]

nodes related:  2265


In [ ]:
# show the format of data
print(len(pre_features))

3816


In [ ]:
from tqdm import tqdm
# Function
#Salton Similarity
def salton_similarity(node1, node2):
    n1 = pre_features[node1]
    n2 = pre_features[node2]
    common_neighors = list(set(n1[2]).intersection(set(n2[2])))
    inter = len(common_neighors)
    degree_out_flow = n1[6]
    degree_in_flow = n2[4]
    
    if inter == 0:
        return 0
    else:
        try:
            sqrt_of_degree = math.sqrt(degree_out_flow * degree_in_flow)
            salton = inter / sqrt_of_degree
            probability = 1 /(1 - math.log(salton)*0.2)
            return probability
        except:
            return 0

#Cosine
def Cosine(Node1, Node2):
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    lm = len(common_neighors)
    if lm == 0:
        return 0
    else:
        return (0.0+lm)/(len(n1[2])*len(n2[2]))

def get_jaccard_coefficient(source, sink):
    """
    in: source::Node object
    in: sink::Node object
    return: jaccard's cofficient::numeric
    """
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
    
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    union_neighbours = neigbours_set_of_source | neigbours_set_of_sink
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
    if len(union_neighbours)==0:
        return 0.0
    return(len(common_neighbours)/len(union_neighbours))

def get_preferential_attachment(source, sink):
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
    
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    
    return len(neigbours_set_of_source)*len(neigbours_set_of_sink)

def get_adamic_adar(source, sink):
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]

    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
    # get the summation
    score = 0
    for common_node in common_neighbours:
        if math.log(len(BasicFeatures[common_node][2])) == 0:
            return 0.0
        score = score + 1/math.log(len(BasicFeatures[common_node][2]))
    return score

def get_resource_allocation(source, sink):
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
#     print(neighbours_of_source_list)
#     print(neighbours_of_sink_list)
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
#     print(common_neighbours)
    score=0
    for common_node in common_neighbours:
        # number of the neighbours of the common_node
        try:
            single_common_node_score = 1/BasicFeatures[common_node][0]
        except:
            single_common_node_score=0
        score = score + single_common_node_score
    return score
    

# how similar are the outbound neighbors of source to sink
# either JA, PA, AA
def get_outbound_similarity_score(source, sink, metric):
    # get the outbound_node of source
    outbound_node_for_source_set = set(BasicFeatures[source][5])
    summation = 0
    for outbound_node_for_source in outbound_node_for_source_set:
        summation =summation + metric(sink,outbound_node_for_source)
    if len(outbound_node_for_source_set) == 0:
        return 0
    score = 1/len(outbound_node_for_source_set)*summation
    return score

# either JA, PA, AA
def get_inbound_similarity_score(source, sink, metric):
    # get the inbound_node of sink
    inbound_node_for_sink_set = set(BasicFeatures[source][3])
    summation = 0
    for inbound_node_for_sink in inbound_node_for_sink_set:
        summation =summation + metric(source,inbound_node_for_sink)
    if len(inbound_node_for_sink_set) == 0:
        return 0
    score = 1/len(inbound_node_for_sink_set)*summation
    return score

def get_common_neighbours(node1, node2):
    try:
        n1 = pre_features[node1]
        n2 = pre_features[node2]
        common_neighors = list(set(n1[2]).intersection(n2[2]))
        return common_neighors
    except:
        return 0

def gen_training_df(final_edges):
    training_df = pd.DataFrame()
    for edge in tqdm(final_edges):
        source = edge[0][0]
        sink = edge[0][1]
        label = edge[1]
        salton_similarity_score = salton_similarity(source, sink)
        cosine = Cosine(source, sink)
        jaccard_coefficient = get_jaccard_coefficient(source, sink)
        preferential_attachment = get_preferential_attachment(source, sink)
        adamic_adar = get_adamic_adar(source, sink)
        resource_allocation = get_resource_allocation(source, sink)

        #salton_similarity_score_out = get_outbound_similarity_score(source, sink, salton_similarity)
        cosine_out = get_outbound_similarity_score(source, sink, Cosine)
        jaccard_coefficient_out = get_outbound_similarity_score(source, sink, get_jaccard_coefficient)
        #preferential_attachment_out = get_outbound_similarity_score(source, sink, get_preferential_attachment)
        #adamic_adar_out = get_outbound_similarity_score(source, sink, get_adamic_adar)
        #resource_allocation_out = get_outbound_similarity_score(source, sink, get_resource_allocation)
        
        
        #salton_similarity_score_in = get_inbound_similarity_score(source, sink, salton_similarity)
        cosine_in = get_inbound_similarity_score(source, sink, Cosine)
        jaccard_coefficient_in = get_inbound_similarity_score(source, sink, get_jaccard_coefficient)
        #preferential_attachment_in = get_inbound_similarity_score(source, sink, get_preferential_attachment)
        #adamic_adar_in = get_inbound_similarity_score(source, sink, get_adamic_adar)
        #resource_allocation_in = get_inbound_similarity_score(source, sink, get_resource_allocation)

        df_row = pd.DataFrame([source, 
                               sink, 
                               label, 
                               salton_similarity_score, 
                               cosine, 
                               jaccard_coefficient,
                               preferential_attachment, 
                               adamic_adar, 
                               resource_allocation,
                               #salton_similarity_score_out,
                               cosine_out,
                               jaccard_coefficient_out,
                               #preferential_attachment_out,
                               #adamic_adar_out,
                               #resource_allocation_out,
                               #salton_similarity_score_in,
                               cosine_in,
                               jaccard_coefficient_in,
                               #preferential_attachment_in,
                               #adamic_adar_in,
                               #resource_allocation_in                         
                              ]).T
        training_df = training_df.append(df_row)
    return training_df

def gen_training_edges(num_of_edges):
    """
    num_of_edges: number of positive edges to generate
    """
    # generate the positive_edge
    ps_edges = random.sample(edges_of_all_test_nodes_related, num_of_edges)
    ps_edges_set = set(ps_edges)   # Here we change ps_edges to hash set to decrease the time complexity
    nodes = set()
    for edge in tqdm(ps_edges):
        nodes.add(edge[0])
        nodes.add(edge[1])
    # generate the negative edges
    count = 0
    final_edges=list()
    while count < num_of_edges:
        if count%1000 ==0:
            print(count)
        node1, node2 = random.sample(nodes,2)
        if (node1,node2) not in ps_edges_set:
            count += 1
            final_edges.append((node1,node2,0))
    for edge in ps_edges:
        final_edges.append((edge[0], edge[1], 1))
    return final_edges

In [ ]:
print(len(training_edges))


64136


In [ ]:
training_df = gen_training_df(training_edges)

100%|██████████| 64136/64136 [01:58<00:00, 541.23it/s]


In [ ]:
save_obj(training_df, 'training_df')
print(training_df[0])

0    1410.0
0    3490.0
0      19.0
0    3947.0
0    1073.0
      ...  
0    3937.0
0     609.0
0     618.0
0    1618.0
0    2814.0
Name: 0, Length: 64136, dtype: float64


In [ ]:
test_edges = []
for elem in test_data:
    test_edges.append((elem[1], elem[2]))

print(test_edges[1], 'length: ', len(test_edges))

(0, 2956) length:  2000


In [ ]:
# 感觉生成test_features 和训练函数差不多，也就是没有label，到时候注意一下
def gen_test_df(test_edges):
    test_df = pd.DataFrame()
    for edge in tqdm(test_edges):
        source = edge[0]
        sink = edge[1]
        #label = edge[2]
        salton_similarity_score = salton_similarity(source, sink)
        cosine = Cosine(source, sink)
        jaccard_coefficient = get_jaccard_coefficient(source, sink)
        preferential_attachment = get_preferential_attachment(source, sink)
        adamic_adar = get_adamic_adar(source, sink)
        resource_allocation = get_resource_allocation(source, sink)

        salton_similarity_score_out = get_outbound_similarity_score(source, sink, salton_similarity)
        cosine_out = get_outbound_similarity_score(source, sink, Cosine)
        jaccard_coefficient_out = get_outbound_similarity_score(source, sink, get_jaccard_coefficient)
        preferential_attachment_out = get_outbound_similarity_score(source, sink, get_preferential_attachment)
        adamic_adar_out = get_outbound_similarity_score(source, sink, get_adamic_adar)
        resource_allocation_out = get_outbound_similarity_score(source, sink, get_resource_allocation)
        
        
        salton_similarity_score_in = get_inbound_similarity_score(source, sink, salton_similarity)
        cosine_in = get_inbound_similarity_score(source, sink, Cosine)
        jaccard_coefficient_in = get_inbound_similarity_score(source, sink, get_jaccard_coefficient)
        preferential_attachment_in = get_inbound_similarity_score(source, sink, get_preferential_attachment)
        adamic_adar_in = get_inbound_similarity_score(source, sink, get_adamic_adar)
        resource_allocation_in = get_inbound_similarity_score(source, sink, get_resource_allocation)

        df_row = pd.DataFrame([source, 
                               sink, 
                               salton_similarity_score, 
                               cosine, 
                               jaccard_coefficient,
                               preferential_attachment, 
                               adamic_adar, 
                               resource_allocation,
                               #salton_similarity_score_out,
                               cosine_out,
                               jaccard_coefficient_out,
                               #preferential_attachment_out,
                               #adamic_adar_out,
                               #resource_allocation_out,
                               #salton_similarity_score_in,
                               cosine_in,
                               jaccard_coefficient_in,
                               #preferential_attachment_in,
                               #adamic_adar_in,
                               #resource_allocation_in                         
                              ]).T
        test_df = test_df.append(df_row)
    return test_df


test_df = gen_test_df(test_edges)

100%|██████████| 2000/2000 [00:03<00:00, 639.92it/s]


In [ ]:
save_obj(test_df, 'test_df')

In [ ]:
# The features for training
final_training_data_df = training_df.iloc[:,3:20]

In [ ]:
print(final_training_data_df)

          3         4         5      6   ...        9         10        11        12
0   0.805935  0.030000  0.115385  100.0  ...  0.012950  0.305202  0.017186  0.066861
0   0.000000  0.000000  0.000000   28.0  ...  0.000000  0.000000  0.100000  0.169231
0   0.876224  0.016234  0.326087  924.0  ...  0.016404  0.270639  0.018688  0.290825
0   0.000000  0.000000  0.000000  261.0  ...  0.000000  0.000000  0.015010  0.155938
0   0.883255  0.019048  0.333333  735.0  ...  0.018075  0.209698  0.011992  0.168904
..       ...       ...       ...    ...  ...       ...       ...       ...       ...
0   0.000000  0.000000  0.000000    7.0  ...  0.000000  0.000000  0.000000  0.000000
0   0.932009  0.053571  0.529412  168.0  ...  0.031193  0.346134  0.035039  0.358137
0   0.000000  0.000000  0.000000   32.0  ...  0.000000  0.000000  0.012861  0.164511
0   0.790092  0.035088  0.100000   57.0  ...  0.057731  0.148440  0.023130  0.284364
0   0.000000  0.000000  0.000000   15.0  ...  0.000000  0.000000 

In [ ]:
# The labels for training
final_labels_df = training_df.iloc[:,2]


In [ ]:
# final_training_data_df = rescale_min_max(final_training_data_df)
X=final_training_data_df
# count=0
# get the data and label
y=final_labels_df

from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X,y)
x_train = X_t
x_test = X_test
y_train = y_t
y_test = y_test

In [ ]:
print(x_train.shape)

(48102, 10)


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
import pickle
import pandas as pd
import time
import csv

In [ ]:
# x_train = np.random.random((1000, 20))
# y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
# x_test = np.random.random((100, 20))
# y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
import tensorflow as tf
model = Sequential()
model.add(Dense(32, input_dim=10, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


# 选择损失函数，优化器
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    #  
    decay_steps = 1000,
    decay_rate = 1,
    staircase = False
)
def get_optimizer():
    return tf.keras.optimizers.Adam(lr_schedule)
opt = get_optimizer()
model.compile(opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=200,
          batch_size=32)
score = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
test_data_df = test_df.iloc[:,2:20]
print(test_df)
print(test_data_df)


        0       1         2         3   ...        8         9         10        11
0      0.0  2917.0  0.000000  0.000000  ...  0.009722  0.049479  0.077691  0.554471
0      0.0  2956.0  0.000000  0.000000  ...  0.000000  0.000000  0.077691  0.554471
0      1.0  4038.0  0.000000  0.000000  ...  0.000034  0.000702  0.042457  0.646066
0      2.0  1848.0  0.775773  0.027778  ...  0.044080  0.117634  0.010460  0.086307
0      3.0   513.0  0.000000  0.000000  ...  0.000000  0.000000  0.034856  0.567946
..     ...     ...       ...       ...  ...       ...       ...       ...       ...
0   3865.0  3924.0  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000
0   3917.0  4025.0  0.000000  0.000000  ...  0.000000  0.000000  0.044608  0.077243
0   3922.0  3947.0  0.000000  0.000000  ...  0.000936  0.015500  0.021753  0.084510
0   3955.0  3987.0  0.760982  0.010811  ...  0.024145  0.087237  0.006939  0.150929
0   3993.0  4063.0  0.000000  0.000000  ...  0.000000  0.000000  0.047750  0

In [ ]:
result = model.predict(test_data_df)

In [ ]:
print(result)
save_obj(result, 'pred1')

[[1.8608219e-03]
 [6.0797250e-04]
 [1.7252525e-02]
 ...
 [3.2354929e-02]
 [9.9852729e-01]
 [4.8601860e-04]]


In [ ]:
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())

def save_prediction_to_csv(result,filename):
    headers = ['id','Predicted']

    with open(filename + str(nowtime()) + ".csv", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(result)
#save_prediction_to_csv(result, "shawn_ann_v1_200epoch")

In [ ]:
ret_res = []
for i in range(1,2001):
  ret_res.append([i, result[i-1][0]])

print(ret_res)

[[1, 0.0018608219], [2, 0.0006079725], [3, 0.017252525], [4, 0.99837875], [5, 0.016958572], [6, 0.050883345], [7, 0.0005948512], [8, 0.99903595], [9, 0.002192472], [10, 0.99883395], [11, 0.0009019055], [12, 0.9987178], [13, 0.0013828605], [14, 0.030999912], [15, 0.9988637], [16, 0.037259027], [17, 0.0018809625], [18, 0.99898094], [19, 0.9989466], [20, 0.9985967], [21, 0.9989466], [22, 0.01691309], [23, 0.6121665], [24, 0.047736656], [25, 0.0011801168], [26, 0.7413626], [27, 0.9971054], [28, 0.99631625], [29, 0.0032401818], [30, 0.1491696], [31, 0.003413955], [32, 0.16586813], [33, 0.6846248], [34, 0.97914314], [35, 0.0015172152], [36, 0.95891595], [37, 0.020639049], [38, 0.002399747], [39, 0.5216821], [40, 0.00063360983], [41, 0.0019537199], [42, 0.83083504], [43, 0.00052409404], [44, 0.99847573], [45, 0.0020142137], [46, 0.9989963], [47, 0.010861858], [48, 0.9990859], [49, 0.9988877], [50, 0.9989736], [51, 0.9951697], [52, 0.0014120155], [53, 0.001325199], [54, 0.0018729718], [55, 0.0

In [ ]:
save_prediction_to_csv(ret_res, "pred3_19_1st")